In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from model import NoteComposeNet
from dataset import MidiDataset, VOCABULARY
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
import librosa 
import torch

In [53]:
df = {
    'notes': [torch.tensor([1 for i in range(0, 255)])],
    'velocities': [torch.tensor([1 for i in range(0, 255)])],
    'durations': [torch.tensor([1 for i in range(0, 255)])],
    'times': [torch.tensor([1 for i in range(0, 255)])],     
}

In [14]:
model = NoteComposeNet()

In [16]:
# Model Specifications
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 0.079MB


In [17]:
midi = MidiDataset(df, context_len = model._context_len)
train_loader = DataLoader(midi, batch_size=1)

data = next(iter(train_loader))
note_tensor = data['notes'].to("cuda")
velocity_tensor = data['velocities'].to("cuda")
duration_tensor = data['durations'].to("cuda")
time_tensor = data['times'].to("cuda")

input = {
    "notes": note_tensor,
    "velocities": velocity_tensor,
    "durations": duration_tensor ,
    "times": time_tensor
}

NameError: name 'df' is not defined

In [19]:
test_inputs = np.random.randint(0, len(VOCABULARY), size=255, dtype=int)
next_notes = model.generate(test_inputs, max_len=100)
print("Note = ", model.detokenize(next_notes))

c:\Users\Joaquin\Desktop\Playground\ML\Composer\model.py:114: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  def detokenize(self, inputs):


Note =  ['A♯-1', 'D♯8', '<EOS>', 'D♯5', 'A1', 'E9', 'A♯0', 'D♯8', 'B2', 'E1', 'A6', 'C♯5', 'G♯3', 'F9', 'C5', 'A6', 'C9', 'B3', 'E1', 'C1', 'C9', 'F5', 'E1', 'A6', 'C9', 'D♯5', 'A1', 'E1', 'D♯5', 'D♯8', 'D♯8', 'B7', 'C♯0', 'F♯9', 'B7', 'F♯1', 'F2', 'D♯5', 'C♯2', 'D7', 'G2', 'A♯0', 'C♯5', 'G5', 'F0', 'D1', 'G♯0', 'E1', 'G♯3', 'E1', 'A1', 'C7', 'F0', 'G0', 'C5', 'F5', 'B3', 'F5', 'E1', 'G2', 'G3', 'A6', 'F2', 'F0', 'B0', 'A6', 'C♯5', 'F♯6', 'C♯2', 'E1', 'A6', 'A1', 'A6', 'A1', 'A6', 'F5', 'G0', 'C5', 'G0', 'A6', 'F5', 'B3', 'D♯-1', 'A6', 'C♯5', 'B3', 'E3', 'A6', 'G0', 'A♯-1', 'F0', 'E0', 'B2', 'E1', 'D2', 'C♯5', 'C♯-1', 'F5', 'E1', 'A6']


# Training

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from model import NoteComposeNet
from dataset import MidiDataset
from torch.utils.data import DataLoader
from train import TrainPipeline

import torch
import pandas as pd

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [2]:
BATCH_SIZE = 32
SAMPLES_PER_TRACK = 128
EPOCHS = 100

In [3]:
model = NoteComposeNet()

In [4]:
CSV_PATH = r'midi-dataset-mini.csv'
df = pd.read_csv(CSV_PATH)
midi = MidiDataset(df, context_len = model._context_len, samples_per_track=SAMPLES_PER_TRACK)
del df

In [5]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=0.001, 
    weight_decay=0.004,
    betas=(0.9, 0.999),
    eps=1e-07, 
    amsgrad=False)
train_loader = DataLoader(
    midi, 
    batch_size=BATCH_SIZE,
    num_workers=1,
    shuffle=True, 
)

424


In [8]:
pipeline = TrainPipeline(train_loader, model, loss_fn, optimizer)
pipeline.train(EPOCHS)

EPOCH 1:


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\Joaquin\miniconda3\envs\agienv\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Joaquin\miniconda3\envs\agienv\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Joaquin\miniconda3\envs\agienv\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "c:\Users\Joaquin\Desktop\Playground\ML\Composer\dataset.py", line 44, in __getitem__
    offset = np.random.randint(0, len(note_slice) - self.context_len)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 